In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import cuml
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv', index_col='id')

In [3]:
train.shape

(750000, 11)

In [4]:
test.shape

(250000, 10)

## Data Understanding

In [5]:
train.sample(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
id,,,,,,,,,,,
227333,Game Day,Episode 59,<NA>,Sports,55.08,Saturday,Afternoon,<NA>,3.0,Neutral,57.89683
429070,Fashion Forward,Episode 52,<NA>,Lifestyle,47.28,Wednesday,Afternoon,18.29,2.0,Neutral,5.08000
405500,Melody Mix,Episode 22,62.46,Music,40.18,Wednesday,Afternoon,49.02,0.0,Neutral,45.83187
514510,Home & Living,Episode 28,98.9,Lifestyle,45.19,Monday,Afternoon,70.23,0.0,Neutral,80.18570
604654,Gadget Geek,Episode 74,78.67,Technology,88.85,Saturday,Afternoon,82.23,0.0,Positive,57.40000
353552,Tech Talks,Episode 1,104.6,Technology,67.14,Tuesday,Afternoon,30.2,3.0,Neutral,65.33834
500057,Mind & Body,Episode 4,<NA>,Health,98.99,Tuesday,Afternoon,<NA>,0.0,Positive,71.72373
455407,Crime Chronicles,Episode 19,114.52,True Crime,87.10,Wednesday,Morning,91.86,1.0,Negative,69.91642
55422,Finance Focus,Episode 34,35.29,Business,58.34,Tuesday,Morning,79.32,2.0,Neutral,12.95421


In [6]:
train.describe()

,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,64.504738,59.859901,52.236449,1.348855,45.437406
std,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,1.300000,0.000000,0.000000,0.000000
25%,35.730000,39.410000,28.380000,0.000000,23.178350
50%,63.840000,60.050000,53.580000,1.000000,43.379460
75%,94.070000,79.530000,76.600000,2.000000,64.811580
max,325.240000,119.460000,119.910000,103.910000,119.970000


In [7]:
train.shape

(750000, 11)

In [8]:
test.shape

(250000, 10)

In [9]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  object
 1   Episode_Title                750000 non-null  object
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object
 6   Publication_Time             750000 non-null  object
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 88.8+ MB


In [10]:
train.isnull().sum()

Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [11]:
test.isnull().sum()

Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

## Data Preparation

Reducing the memory space of the dataset by converting the num_cols into float32 and cat_cols into category

In [12]:
num_cols = test.select_dtypes(include=['float64']).columns
for col in num_cols:
    train[col] = train[col].astype('float32')
    test[col] = test[col].astype('float32')

In [13]:
cat_cols = train.select_dtypes(include=['object']).columns
for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [14]:
from scipy.stats.mstats import winsorize

train['Listening_Time_minutes'] = winsorize(train['Listening_Time_minutes'], limits=(0.01, 0.01))

In [15]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   Podcast_Name                 750000 non-null  category
 1   Episode_Title                750000 non-null  category
 2   Episode_Length_minutes       662907 non-null  float32
 3   Genre                        750000 non-null  category
 4   Host_Popularity_percentage   750000 non-null  float32
 5   Publication_Day              750000 non-null  category
 6   Publication_Time             750000 non-null  category
 7   Guest_Popularity_percentage  603970 non-null  float32
 8   Number_of_Ads                749999 non-null  float32
 9   Episode_Sentiment            750000 non-null  category
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: category(6), float32(4), float64(1)
memory usage: 27.5 MB


In [16]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='mean')
num_cols = test.select_dtypes(include=['float32']).columns
train[num_cols] = impute.fit_transform(train[num_cols])
test[num_cols] = impute.fit_transform(test[num_cols])

In [17]:
sentiments = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
train['Episode_Sentiment'] = train['Episode_Sentiment'].map(sentiments).astype('float32')
test['Episode_Sentiment'] = test['Episode_Sentiment'].map(sentiments).astype('float32')

## Feature engineering

## Count Encoding for high cardinality columns

In [18]:
from category_encoders import CountEncoder

ce = CountEncoder()
count_category = ['Podcast_Name', 'Genre', 'Episode_Title']
train[count_category] = ce.fit_transform(train[count_category])
test[count_category] = ce.transform(test[count_category])

In [19]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Publication_Day', 'Publication_Time']
for colname in cat_cols:
    le = LabelEncoder()
    train[colname] = le.fit_transform(train[colname])
    test[colname] = le.transform(test[colname])

In [20]:
from sklearn.preprocessing import KBinsDiscretizer

kmeans = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
train['Kmeans_bins'] = kmeans.fit_transform(train[['Episode_Length_minutes']])
test['Kmeans_bins'] = kmeans.transform(test[['Episode_Length_minutes']])

In [21]:
for col in ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Length_minutes']:
    for k in range(0, 3):
        new_round_col = f"{col}_round{k}"
        train[new_round_col] = train[col].round(k)
        test[new_round_col] = test[col].round(k)

In [22]:
train['Guest_Popularity_percentage'] =  train['Guest_Popularity_percentage'].clip(upper=100)
train['Host_Popularity_percentage'] =  train['Host_Popularity_percentage'].clip(upper=100)
test['Guest_Popularity_percentage'] =  test['Guest_Popularity_percentage'].clip(upper=100)
test['Host_Popularity_percentage'] =  test['Host_Popularity_percentage'].clip(upper=100)


In [23]:
train['Average_popularity'] = (train['Host_Popularity_percentage'] + train['Guest_Popularity_percentage']) / 2
test['Average_popularity'] = (test['Host_Popularity_percentage'] + test['Guest_Popularity_percentage']) / 2
train['minutes_per_ads'] = train['Episode_Length_minutes'] / (train['Number_of_Ads'] + 1e-3) 
test['minutes_per_ads'] = test['Episode_Length_minutes'] / (test['Number_of_Ads'] + 1e-3)

In [24]:
train.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes', 'Kmeans_bins',
       'Host_Popularity_percentage_round0',
       'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2',
       'Guest_Popularity_percentage_round0',
       'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0',
       'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2',
       'Average_popularity', 'minutes_per_ads'],
      dtype='object')

In [25]:
features = ['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre', 'Publication_Day', 'Publication_Time',
       'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Kmeans_bins', 'Host_Popularity_percentage_round0', 'Host_Popularity_percentage_round1',
       'Host_Popularity_percentage_round2', 'Guest_Popularity_percentage_round0', 'Guest_Popularity_percentage_round1',
       'Guest_Popularity_percentage_round2', 'Episode_Length_minutes_round0', 'Episode_Length_minutes_round1', 'Episode_Length_minutes_round2', 'Average_popularity']

In [26]:
interact = []
for i, c1 in enumerate(features):
    for c2 in (features[i+1:]):
        n = f'{c1}_{c2}'
        train[n] = train[c1] * train[c2]
        test[n] = test[c1] * test[c2]
        interact.append(n)

print(f"There are {len(interact)} interaction features:")

There are 210 interaction features:


In [27]:
from sklearn.model_selection import KFold
from cuml.metrics import mean_squared_error
from xgboost import XGBRegressor


y = train.Listening_Time_minutes
n_folds = 7

test_preds = np.zeros(len(test))
oof_xgb = np.zeros(len(y))

stats = ['mean', 'median', 'std', 'skew']
quantiles = [5, 10, 25, 40, 50, 60, 75, 90, 95]


kf = KFold(n_splits=n_folds, shuffle=True, random_state=34)

for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
    X_train, X_valid = train.iloc[train_idx][features], train.iloc[valid_idx][features]
    y_train, y_valid = train.iloc[train_idx]['Listening_Time_minutes'], train.iloc[valid_idx]['Listening_Time_minutes']

    X_test = test[features].copy()
    
    train_with_target = X_train.copy()
    train_with_target['Listening_Time_minutes'] = y_train
    
    for q in quantiles:
        result = train_with_target.groupby('Episode_Length_minutes')['Listening_Time_minutes'] \
            .quantile(q/100).reset_index()
        result.rename(columns={'Listening_Time_minutes': f'quantile_{q}'}, inplace=True)
        X_train = X_train.merge(result, on='Episode_Length_minutes', how='left')
        X_train[f'quantile_{q}'] = X_train[f'quantile_{q}'].fillna(0.0).astype('float32').fillna(0.0)
        X_valid = X_valid.merge(result, on='Episode_Length_minutes', how='left').astype('float32').fillna(0.0)
        X_test = X_test.merge(result, on='Episode_Length_minutes', how='left').astype('float32').fillna(0.0)

    grouped_stats = train_with_target.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
    grouped_stats.columns = [f'TE1_WC_{stat}' for stat in stats]

    grouped_stats_full = train.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
    grouped_stats_full.columns = [f'TE1_WC_{stat}' for stat in stats]

    X_train = X_train.merge(grouped_stats, on='Episode_Length_minutes', how='left').astype('float32')
    X_valid = X_valid.merge(grouped_stats_full, on='Episode_Length_minutes', how='left').astype('float32')
    X_test = X_test.merge(grouped_stats_full, on='Episode_Length_minutes', how='left').astype('float32')
    
    xgb = XGBRegressor(
        n_estimators=500, 
        learning_rate=0.07, 
        reg_lambda=8.20664, 
        max_depth=11,
        colsample_bytree=0.5,
        subsample=0.8,
        device='cuda',
        tree_method='gpu_hist',
        eval_metric='rmse',
        verbosity=0,
        random_state=34
    )
    xgb.fit(X_train, y_train)
    
    valid_pred = xgb.predict(X_valid)
    fold_rmse = np.sqrt(mean_squared_error(y_valid, valid_pred))
    print(f"Fold {fold + 1} RMSE: {fold_rmse:.2f}")
    
    test_preds += xgb.predict(X_test) / n_folds


Fold 1 RMSE: 12.72
Fold 2 RMSE: 12.62
Fold 3 RMSE: 12.60
Fold 4 RMSE: 12.52
Fold 5 RMSE: 12.59
Fold 6 RMSE: 12.59
Fold 7 RMSE: 12.67


## Final Submission

In [28]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv')
sub['Listening_Time_minutes'] = test_preds
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
